## Model Evaluation

In [1]:
import os

In [2]:
os.chdir("../")

In [ ]:
os.path.dirname()

'c:\\Users\\Tyson.Barreto.AAT\\OneDrive - Association of Accounting Technicians\\Desktop\\Learning\\Learning\\MLOps\\MLOps-CNNClassifier'

In [35]:
import tensorflow as tf
from pathlib import Path
from cnnClassifier.utils import read_yaml


model = tf.keras.models.load_model(r".\artifacts\training\model.h5")


In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size:list
    params_batch_size:int

In [29]:
#from cnnClassifier.entity import EvaluationConfig
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH 
from cnnClassifier.utils import read_yaml, save_json, create_directories


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self)->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = r".\artifacts\training\model.h5",
            training_data = r".\artifacts\data_ingestion\PetImages",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config


In [37]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config
        
    
    def valid_generator(self):
        datagenerator_kwargs = dict(
            rescale =1./255,
            validation_split=0.2
        )
        
        dataflow_kwargs= dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation='bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    
        

In [38]:
try:
    config= ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2024-09-11 10:18:20,536: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-09-11 10:18:20,541: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-09-11 10:18:20,544: INFO: common: created directory at: artifacts:]
Found 78 images belonging to 2 classes.
5/5 [==============================] - 18s 4s/step - loss: 21.2826 - accuracy: 0.5000
[2024-09-11 10:18:39,017: INFO: common: json file saved at: scores.json:]
